# copy of generate_error.py from proPTV

In [17]:
# import matplotlib.pyplot as plt
import numpy as np 
from pathlib import Path


from optv.parameters import ControlParams, VolumeParams
from optv.calibration import Calibration
# from optv.orientation import external_calibration, full_calibration

from optv.imgcoord import image_coordinates
from optv.transforms import convert_arr_metric_to_pixel, convert_arr_pixel_to_metric, distorted_to_flat
from optv.orientation import point_positions
from optv.correspondences import MatchedCoords
from optv.tracking_framebuf import TargetArray

# import plotly.express as px
import plotly.figure_factory as ff
# import plotly.graph_objects as go
from scipy.optimize import minimize

import pandas as pd
import plotly.express as px

In [18]:

# copy of SaveLoad.py
# import numpy as np

def LoadMarkerList(cam):
    return np.loadtxt('markers_c'+str(cam)+'.txt')

def SaveMarkerList(data):
    # Format: x0, y0, x1, y1, x2, y2, x3, y3, X, Y, Z, dX, dY, dZ
    return np.savetxt('markers_error.txt',data)



In [19]:

class Parameter:
    cams = [0,1,2,3]
    Vmin = [0,0,0]
    Vmax = [300,300,300]
    N1, N2 = 361, 5

# load parameter
params = Parameter()



In [20]:
cases_path = (Path.cwd().parent / 'cases')
cases_path.exists()

True

In [21]:
cases = list(cases_path.rglob('case_*'))
order = [1,0,2]
cases =  [cases[i] for i in order ]

In [22]:
for c in cases:
    list_files = list(c.rglob('markers*'))
    list_files.sort()
    print(list_files)


[PosixPath('/home/user/Documents/repos/proPTV_OpenPTV_MyPTV_comparison/cases/case_allmarkers/markers_c0.txt'), PosixPath('/home/user/Documents/repos/proPTV_OpenPTV_MyPTV_comparison/cases/case_allmarkers/markers_c1.txt'), PosixPath('/home/user/Documents/repos/proPTV_OpenPTV_MyPTV_comparison/cases/case_allmarkers/markers_c2.txt'), PosixPath('/home/user/Documents/repos/proPTV_OpenPTV_MyPTV_comparison/cases/case_allmarkers/markers_c3.txt')]
[PosixPath('/home/user/Documents/repos/proPTV_OpenPTV_MyPTV_comparison/cases/case_interpolation/markers_c0.txt'), PosixPath('/home/user/Documents/repos/proPTV_OpenPTV_MyPTV_comparison/cases/case_interpolation/markers_c1.txt'), PosixPath('/home/user/Documents/repos/proPTV_OpenPTV_MyPTV_comparison/cases/case_interpolation/markers_c2.txt'), PosixPath('/home/user/Documents/repos/proPTV_OpenPTV_MyPTV_comparison/cases/case_interpolation/markers_c3.txt')]
[PosixPath('/home/user/Documents/repos/proPTV_OpenPTV_MyPTV_comparison/cases/case_extrapolation/markers_c0

In [23]:
def array_to_calibration(x:np.ndarray, cal:Calibration) -> None:
    cal.set_pos(x[:3])
    cal.set_angles(x[3:6])
    cal.set_primary_point(x[6:9])
    cal.set_radial_distortion(x[9:12])
    cal.set_decentering(x[12:14])
    cal.set_affine_trans(x[14:])
    return None

def calibration_to_array(cal:Calibration) -> np.ndarray:
    return np.concatenate([
        cal.get_pos(),
        cal.get_angles(),
        cal.get_primary_point(),
        cal.get_radial_distortion(),
        cal.get_decentering(),
        cal.get_affine(),
    ])

def error_function(x, cal, XYZ, xy, cpar):
    
    array_to_calibration(x, cal)

    targets = convert_arr_metric_to_pixel(
        image_coordinates(XYZ, cal, cpar.get_multimedia_params()),
    cpar,
    )
    # err = np.sum(np.abs(xy - targets))
    err = np.sum((xy - targets)**2)
    # print(err)
    return err

def targetize(detects, approx_size, sumg=10):
    """
    Creates a correct TargetArray object with the detected positions and some
    placeholder values for target parameters that I don't use.
    
    Arguments:
    detects - (n,2) array, pixel coordinates of a detected target.
    approx_size - a value to use for the pixel size placeholders.
    sumg - a value to use for the sum of grey values placeholder.
        Default: 10.
    """
    targs = TargetArray(len(detects))
    
    tnum = 0
    for t, pos in zip(targs, detects):
        t.set_pos(pos)
        t.set_pnr(tnum)
        t.set_sum_grey_value(sumg) # whatever
        t.set_pixel_counts(approx_size**2 * 4, approx_size*2, approx_size*2)
        t.set_tnr(-1) # The official "correspondence not found" that 
                               # the rest of the code expects.
        tnum += 1
    
    return targs

def pixel_to_3d(markers, cpar, cals, vpar):
    """ converts numpy array of size (2,) from pixel to flat coordinates"""
    detected = []
    corrected = []
    pnrs = []
    for cix in range(cpar.get_num_cams()):
        targs = targetize(markers[cix][:,:2], 1,1)
        # targs.sort_y()  # not sure why it matters but it does
        
        detected.append(targs)
        pnrs.append([t.pnr() for t in targs])

        # mc = 
        # _, pnr = mc.as_arrays()
        # pnrs.append(pnr)
        corrected.append(MatchedCoords(targs, cpar, cals[cix]))

    flat = np.array([corrected[cix].get_by_pnrs(np.array(pnrs[cix])) \
            for cix in range(len(cals))])

    pos3d, rcm = point_positions(flat.transpose(1,0,2), cpar, cals, vpar)

    return pos3d, rcm

In [24]:
n_cams = len(params.cams)

cpar = ControlParams(n_cams)
cpar.read_control_par(b"parameters/ptv.par")

vpar = VolumeParams()
vpar.read_volume_par(b"parameters/criteria.par")

# Calibration initial guess 

cals = []
for i_cam in range(n_cams):
    cal = Calibration()
    tmp = cpar.get_cal_img_base_name(i_cam)
    cal.from_file(tmp + b".ori", tmp + b".addpar")
    print(cal.get_pos(), cal.get_angles())
    cals.append(cal)



for case in cases:
    # if 'allmarkers' not in case.name:
    #     continue

    list_files = list(case.rglob('markers*'))
    list_files.sort()
    case_name = case.name.split('case_')[-1] # 'interpolation', ...
    print(case_name)
    # print([_.name for _ in list_files])
    
    # load marker
    markers = [np.loadtxt(_) for _ in list_files]

    if 'allmarkers' in case.name:
        all_markers = markers


    for c in params.cams:
        # print(f" Camera {c}\n")

        XYZ = markers[c][:,2:]
        xy = markers[c][:,:2]
        ID = np.argwhere((XYZ[:,0]>-1))[:,0]

        cal = cals[c]
        # print what you get to see it's still a valid guess
        cal.get_pos(), cal.get_angles()

        
        # We could use this step only if we do not have a good
        # initial guess, but we have one from the previous step

        
        four_points = xy[[0,int(ID.max()/4),int(ID.max()*3/4),ID.max()],:] # choose manually
        ref_pts = XYZ[[0,int(ID.max()/4),int(ID.max()*3/4),ID.max()],:]


        targets = convert_arr_metric_to_pixel(
            image_coordinates(ref_pts, cal, cpar.get_multimedia_params()),
        cpar,
        )
        print(f"Before: {four_points - targets}")


        # external_calibration(cal, ref_pts, four_points, cpar)


        x0 = calibration_to_array(cal)
        # print(x0)
        sol = minimize(error_function, x0, args=(cal, XYZ, xy, cpar), method='Nelder-Mead', tol=1e-11)

        array_to_calibration(sol.x, cal)

        targets = convert_arr_metric_to_pixel(
            image_coordinates(ref_pts, cal, cpar.get_multimedia_params()),
        cpar,
        )
        print(f"After: {four_points - targets}")

        targets = convert_arr_metric_to_pixel(
            image_coordinates(XYZ, cal, cpar.get_multimedia_params()),
        cpar,
        )

        # px.scatter(x=xy[:,0], y=xy[:,1], color=ID).show()
        # fig = ff.create_quiver(x=xy[:,0], y=xy[:,1], u=targets[:,0]-xy[:,0], v=targets[:,1]-xy[:,1], scale=5)
        # fig.show()

        # Not sure I understand it correctly, we calibrate with 
        # some markers but always compare with the full set


    # Note that we always use allmarkers for comparison:
    # newXYZ, rcm = pixel_to_3d(all_markers, cpar, cals, vpar)


    XYZ = all_markers[0][:,2:]
    ID = np.argwhere((XYZ[:,0]>-1))[:,0]

    newXYZ, rcm = pixel_to_3d(all_markers, cpar, cals, vpar)
    errors = newXYZ - XYZ

    print(f" Error rms: {np.sqrt(np.sum(errors**2))}")

    # print(rcm)

    newxyz = pd.DataFrame(XYZ, columns=['x','y','z'])
    newxyz['id'] = ID
    px.scatter_3d(x=newxyz['x'], y=newxyz['y'], z=newxyz['z'], color=newxyz['id']).show()

    
    newxyz = pd.DataFrame(newXYZ, columns=['x','y','z'])
    newxyz['id'] = range(len(newXYZ))
    px.scatter_3d(x=newxyz['x'], y=newxyz['y'], z=newxyz['z'], color=newxyz['id']).show()

    # get 3D positions from plate
    # # triangulate 3D position
    # camPs = [ np.asarray([markers[0][i,:2],markers[1][i,:2],markers[2][i,:2],markers[3][i,:2]]) for i in range(len(markers[0]))]
    # P = np.asarray([NewtonSoloff_Triangulation(setP, ax, ay, params)[0] for setP in camPs])

    # # compare error
    # error = XYZ-P
    # output = np.zeros([361*5,8+3+3])
    # for i in range(len(params.cams)):
    #     output[:,2*i:2*(i+1)] = np.loadtxt('markers_c'+str(params.cams[i])+'.txt')[:,:2]
    # output[:,8:11] = XYZ
    # output[:,11:] = error
    # # SaveMarkerList(output)

    # err = np.linalg.norm(error,axis=1)
    # mean_err = np.mean(err)
    # std_err = np.std(err)
    # print('MAE: ', mean_err, ' +- ' , std_err )
    # import matplotlib.pyplot as plt

    # axis = plt.figure().add_subplot(111, projection='3d')
    # axis.scatter(P[:,2], P[:,0], P[:,1],c='red',s=5)
    # axis.scatter(XYZ[ID,2], XYZ[ID,0], XYZ[ID,1],c='black',s=5)
    # axis.set_xlabel('Z',fontsize=13), axis.set_ylabel('X',fontsize=13), axis.set_zlabel('Y',fontsize=13)
    # plt.tight_layout()
    # plt.show()

    # axis = plt.figure().add_subplot(111, projection='3d')
    # axis.quiver(P[:,2],P[:,0],P[:,1], P[:,2]-XYZ[ID,2], P[:,0]-XYZ[ID,0], P[:,1]-XYZ[ID,1],color='red',length=10,normalize=True)
    # axis.set_xlabel('Z',fontsize=13), axis.set_ylabel('X',fontsize=13), axis.set_zlabel('Y',fontsize=13)
    # plt.tight_layout()
    # plt.show()

[ 322.38251339    4.4040676  1031.76370716] [ 0.1884708   0.20904414 -0.03438548]
[-1.72196260e-01  2.96220478e+02  1.03049543e+03] [-0.2028321  -0.18304368 -0.02799897]
[ 323.84409902  298.102586   1028.23341819] [-0.19496146  0.21075588  0.0246242 ]
[-1.32199240e-01  1.49476157e+01  1.02278223e+03] [ 0.17005024 -0.1784537   0.00354099]
allmarkers
Before: [[-3.19232797 -0.09974858]
 [-0.33759279 -5.88781762]
 [-0.77408411 -0.08001957]
 [ 5.05276383  0.65211193]]
After: [[-2.6215738   3.63336465]
 [ 0.54359133 -0.53934995]
 [ 0.49108171  0.06665196]
 [ 0.53445444  2.37726721]]
Before: [[ 4.98487101 -8.00833928]
 [-0.35313061 -1.63648251]
 [-0.78644099  0.22881962]
 [ 7.06044941  1.57179241]]
After: [[ 2.11565888  0.08613557]
 [-0.12295455 -1.38298052]
 [-0.28934187 -0.26649361]
 [ 3.52230071  2.02476222]]
Before: [[ 1.90844903 -6.76866576]
 [ 0.16652637 -3.44572205]
 [-2.17379171  2.02534687]
 [ 9.0635969   1.22556345]]
After: [[ 0.84521945  0.81413058]
 [ 0.77585499 -0.81026266]
 [-0.

interpolation
Before: [[-2.6215738   3.63336465]
 [-1.09364832 -0.05676841]
 [-0.38049824  0.51165145]
 [ 0.53445444  2.37726721]]
After: [[ 0.51941838  0.92849213]
 [-0.68043669  0.18606609]
 [-0.47120839  0.39083558]
 [ 1.39843087  2.2565011 ]]
Before: [[ 2.11565888  0.08613557]
 [ 1.15527469  0.81502088]
 [-0.03621904  1.02611287]
 [ 3.52230071  2.02476222]]
After: [[ 2.4550568  -0.39867499]
 [ 0.91598795  0.48669355]
 [-0.16956568  0.93367702]
 [ 2.78719193  2.34585582]]
Before: [[ 0.84521945  0.81413058]
 [-0.68209423  1.08334204]
 [ 0.19546704  0.89889284]
 [ 2.79771961 -0.163284  ]]
After: [[ 0.80896816  0.50438289]
 [-0.39509213  0.85273553]
 [ 0.09621675  0.47560145]
 [ 0.86741949  1.57186189]]
Before: [[ 5.23320359 -0.8083911 ]
 [ 3.39758403  2.17151901]
 [-0.89194673  1.72940644]
 [ 1.56478907 -1.26417491]]
After: [[ 4.43600669 -1.77515006]
 [ 0.67925135  1.40540517]
 [-0.63301418  1.03558538]
 [ 0.55517494  0.4926861 ]]
 Error rms: 16.263008636562702


extrapolation
Before: [[ 1.13363479  0.91951167]
 [ 0.72366045  0.17361343]
 [-0.44077265  0.54220063]
 [ 0.36462638  1.82096545]]
After: [[ 0.31856665  0.90120912]
 [-0.19462721 -0.17404395]
 [-0.20744451  0.58806722]
 [ 0.67161988  1.4105159 ]]
Before: [[ 1.59785682 -0.41871404]
 [ 0.28758324  0.07616505]
 [-1.04890667  0.01405838]
 [ 0.02675926  1.23742935]]
After: [[ 1.88166174  0.0676864 ]
 [ 0.50327686  0.50359675]
 [-0.79994552  0.17188255]
 [ 0.28319079  1.47961407]]
Before: [[ 2.1181972  -1.01894062]
 [ 1.14494786 -0.79691334]
 [ 0.12482705 -0.56820996]
 [ 0.21323175  0.52988931]]
After: [[ 1.67375106 -0.55689126]
 [ 0.15696156 -0.54207741]
 [ 0.33648607 -0.22362064]
 [ 0.26001402  0.66792833]]
Before: [[ 2.96749968 -1.66538916]
 [-0.33141263  1.15486514]
 [-1.00766928  0.91108453]
 [ 0.18314691  0.00950385]]
After: [[ 1.17223072 -0.12599897]
 [-0.41018204  0.39974094]
 [-0.49418909  0.83284197]
 [ 0.64880427  0.53079295]]
 Error rms: 19.88617756087613
